In [ ]:
import tqdm
import pandas as pd
import pickle as pkl

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
restoredUsersMap= {}
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersMap[i]= 1

In [ ]:
with open(f'../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user_id= i['user']['id_str']
                if restoredUsersMap.get(user_id):
                    restoredUsersMap[user_id]= i['user']['screen_name']
    except EOFError:
        pass

In [ ]:
file1= open("./restoredUsersMap.txt", "w+")
file1.writelines([f'{restoredUsersMap[i]}\n' for i in restoredUsersMap])
file1.close()

In [ ]:
def getStats(df):
    print('Number of Users, Number of Tweets')
    print(len(df['user.id_str'].unique()), len(df.index))

In [ ]:
def getPickleLoaderStats(pklFile):
    global totalNormal, usersNormal
    try:
        while True:
            temp= pkl.load(pklFile)
            for i in tqdm.tqdm(temp):
                usersNormal.add(i['user']['id_str'])
            totalNormal+= len(temp)
    except EOFError:
        pass

In [ ]:
def getPickleLoaderStatsSuspended(pklFile, listUsers):
    global totalSuspended, usersSuspended, totalRestored, usersRestored
    try:
        while True:
            temp= pkl.load(pklFile)
            for i in tqdm.tqdm(temp):
                user_id= i['user']['id_str']
                if  user_id in listUsers:
                    usersRestored.add(user_id)
                    totalRestored+= 1
                else:
                    usersSuspended.add(user_id)
                    totalSuspended+= 1
    except EOFError:
        pass

In [ ]:
df= pd.DataFrame(loadPickle('Regular_After_py37.pkl'))
df2= pd.DataFrame(loadPickle('Regular_Before_py37.pkl'))
df2= df2[df2['user.id_str'].isin(df['user.id_str'])].copy()
df= df[df['user.id_str'].isin(df2['user.id_str'])].copy()

In [ ]:
regularTimeline= pd.concat([df, df2], ignore_index= True)
getStats(regularTimeline)

In [ ]:
regularList= set(list(regularTimeline['user.id_str'].unique()))
df, df2, regularTimeline= None, None, None

In [ ]:
totalRegularBeforeElectionForAfter= {}

In [ ]:
with open(f'../data/all_tweets_normal_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user_id= i['user']['id_str']
                if user_id in regularList:
                    if totalRegularBeforeElectionForAfter.get(user_id):
                        totalRegularBeforeElectionForAfter[user_id]+= 1
                    else:
                        totalRegularBeforeElectionForAfter[user_id]= 1
    except EOFError:
        pass

In [ ]:
sum([totalRegularBeforeElectionForAfter[i] for i in totalRegularBeforeElectionForAfter])

In [ ]:
df= pd.DataFrame(loadPickle('Restored_After_py37.pkl'))
df2= pd.DataFrame(loadPickle('Restored_Before_py37.pkl'))
df2= df2[df2['user.id_str'].isin(df['user.id_str'])].copy()
df= df[df['user.id_str'].isin(df2['user.id_str'])].copy()

In [ ]:
restoredTimeline= pd.concat([df, df2], ignore_index= True)
getStats(restoredTimeline)

In [ ]:
restoredList= set(list(restoredTimeline['user.id_str'].unique()))
df, df2, restoredTimeline= None, None, None

In [ ]:
totalRestoredBeforeElectionForAfter= {}

In [ ]:
with open(f'../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user_id= i['user']['id_str']
                if user_id in restoredList:
                    if totalRestoredBeforeElectionForAfter.get(user_id):
                        totalRestoredBeforeElectionForAfter[user_id]+= 1
                    else:
                        totalRestoredBeforeElectionForAfter[user_id]= 1
    except EOFError:
        pass

In [ ]:
sum([totalRestoredBeforeElectionForAfter[i] for i in totalRestoredBeforeElectionForAfter])

In [ ]:
totalNormal, usersNormal= 0, set()
with open(f'../data/all_tweets_normal_jan3_2021.pickle', 'rb') as f:
    getPickleLoaderStats(f)
print(totalNormal, len(usersNormal))

In [ ]:
totalSuspended, totalRestored, usersSuspended, usersRestored= 0, 0, set(), set()
with open(f'../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    getPickleLoaderStatsSuspended(f, loadTxt('unsuspended_8_Oct.txt'))
print(totalSuspended, len(usersSuspended), totalRestored, len(usersRestored))